## Project I

### CLASSIFICATION OF IMAGES –INTEL DATASET Using MLP(Multi Layer Perceptron)

In [1]:
#Importing essential libraries
from sklearn.datasets import fetch_openml

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os 
import cv2
import random 
import math

from datetime import datetime

from sklearn import metrics
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

import tensorflow as tf
from tensorflow import keras

In [ ]:
#Input the Directory path where the datasets(seg_train and seg_test) are stored
DATADIRTRAIN ="C:\Datasets\seg_train" #path for seg_train
DATADIRTEST="C:\Datasets\seg_test"    #path for seg_test
CATEGORIES= ["street","forest","mountain","sea","glacier","buildings"]

In [ ]:
#Visualizing the data
for category in CATEGORIES:
    path =os.path.join(DATADIRTRAIN,category)
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path,img),cv2.IMREAD_COLOR)
        plt.imshow(img_array)
        plt.show()
        break
    break

In [ ]:
#loading the training data
training_data = []   
IMG_SIZE = 100    # Resizing the Image to 100x100 pixels

def create_training_data():
    for category in CATEGORIES:
        path =os.path.join(DATADIRTRAIN,category)
        class_num= CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path,img),cv2.IMREAD_GRAYSCALE) #Conversion to Grayscale
                new_array = cv2.resize(img_array,(IMG_SIZE , IMG_SIZE)) 
                training_data.append([new_array,class_num])
            except Exception as e:
                pass
            
create_training_data()
print(len(training_data))
random.shuffle(training_data)  #Preprocessing Technique: Performing random shuffling of training data

In [ ]:
#Creating inputs and labels of training data
X_train=[]
y_train=[]

for features, label in training_data:
    X_train.append(features)
    y_train.append(label)
X_train=np.array(X_train)
y_train=np.array(y_train)
print(X_train.shape)
print(y_train.shape)          

In [ ]:
#Preprocessing Technique: Reshaping Training dataset
X_train = np.array(X_train).reshape(len(training_data), -1)
print(X_train.shape)
y_train = np.array(y_train).reshape(-1,1)
print(y_train.shape)

In [ ]:
#loading the testing data
testing_data = []
IMG_SIZE=100   #Preprocessing technique: Resizing the Image to 100x100 pixels

def create_testing_data():
    for category in CATEGORIES:
        path =os.path.join(DATADIRTEST,category)
        class_num= CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path,img),cv2.IMREAD_GRAYSCALE) #Conversion to Grayscale
                new_array = cv2.resize(img_array,(IMG_SIZE , IMG_SIZE)) 
                testing_data.append([new_array,class_num])
            except Exception as e:
                pass
            
create_testing_data()
print(len(testing_data))
random.shuffle(testing_data)

In [ ]:
#Creating inputs and labels of testing data
X_test=[]
y_test=[]

for features, label in testing_data:
    X_test.append(features)
    y_test.append(label)
X_test=np.array(X_test)
y_test=np.array(y_test)
print(X_test.shape)
print(y_test.shape)

In [ ]:
#Preprocessing Technique: Reshaping Testing dataset
X_test= np.array(X_test).reshape(len(testing_data),-1)
print(X_test.shape)
y_test=np.array(y_test).reshape(-1,1)
print(y_test.shape)

In [ ]:
#Cross Validation Technique(Validation set Approach)
X_train, X_val, y_train, y_val  = train_test_split(X_train, y_train, test_size = 0.2, random_state = 42)
print(X_train.shape, X_val.shape)
print(y_train.shape, y_val.shape)

#Stratified Kfold
#skf=StratifiedKFold(n_splits=6, shuffle=True, random_state=42)
#skf.get_n_splits(X_train,y_train)
#for train_index, Val_index in skf.split(X_train,y_train):
#    print("TRAIN:", train_index, "TEST:", Val_index)
#    X_train_s, X_val_s = X_train[train_index], X_train[Val_index]
#    y_train_s, y_val_s = y_train[train_index], y_train[Val_index]

#Kfold
#kf=KFold(n_splits=3, shuffle=True, random_state=42)
#kf.get_n_splits(X_train)
#for train_index, Val_index in kf.split(X_train):
#    print("TRAIN:", train_index, "TEST:", Val_index)
#    X_train_s, X_val_s = X_train[train_index], X_train[Val_index]
#    y_train_s, y_val_s = y_train[train_index], y_train[Val_index]

#LOOCV
#kf=KFold(n_splits=X_train.shape[0], shuffle=True, random_state=42)
#kf.get_n_splits(X_train)
#for train_index, Val_index in kf.split(X_train):
#    print("TRAIN:", train_index, "TEST:", Val_index)
#    X_train_s, X_val_s = X_train[train_index], X_train[Val_index]
#    y_train_s, y_val_s = y_train[train_index], y_train[Val_index]

In [ ]:
#Adaptive Learning Rate(ALR) Model Improvements
#Step Decay of learning rate 
from tensorflow.keras.callbacks import LearningRateScheduler
def step_decay(epoch):
   initial_lrate = 0.1 #Setting the initial learning rate to 0.1 
   drop = 0.5
   epochs_drop = 10.0  #Drops the learning rate to half after every 10 epochs
   lrate = initial_lrate * math.pow(drop,  
           math.floor((1+epoch)/epochs_drop))
   return lrate
lrate = LearningRateScheduler(step_decay)

In [ ]:
#Using Keras to build and train the neural network model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import TensorBoard

#model 10000-3000-2000-6
model = keras.models.Sequential()
model.add(keras.layers.Dense(3000, activation ="relu"))
model.add(keras.layers.Dense(2000, activation ="relu"))
model.add(keras.layers.Dropout(0.5))   #Performing Dropout with 0.5 dropout rate
model.add(keras.layers.Dense(6, activation = "softmax"))

#model 10000-8700-8700-6
#model = keras.models.Sequential()
#model.add(keras.layers.Dense(8700, activation ="relu"))
#model.add(keras.layers.Dense(8700, activation ="relu"))
#model.add(keras.layers.Dropout(0.5))
#model.add(keras.layers.Dense(6, activation = "softmax"))

#model 10000-2000-300-300-128-6
#model = keras.models.Sequential()
#model.add(keras.layers.Dense(2000, activation ="relu"))
#model.add(keras.layers.Dense(300, activation ="relu"))
#model.add(keras.layers.Dropout(0.5))
#model.add(keras.layers.Dense(300, activation ="relu"))
#model.add(keras.layers.Dropout(0.5))
#model.add(keras.layers.Dense(128, activation ="relu"))
#model.add(keras.layers.Dropout(0.5))
#model.add(keras.layers.Dense(6, activation = "softmax"))

#Trying Different Activation Function in the hidden layer
#model = keras.models.Sequential()
#model.add(keras.layers.Dense(3000, activation ="sigmoid"))
#model.add(keras.layers.Dense(2000, activation ="sigmoid"))
#model.add(keras.layers.Dense(6, activation = "softmax"))

#model = keras.models.Sequential()
#model.add(keras.layers.Dense(3000, activation ="relu"))
#model.add(keras.layers.Dense(2000, activation ="relu"))
#model.add(keras.layers.Dense(6, activation = "sigmoid"))


#Data scaling- for cross validation set approach
X_train = X_train.astype('float32')/255.0
y_train = y_train.astype('uint')
X_val = X_val.astype('float32')/255.0
y_val = y_val.astype('uint')

#for Cross validation Kfold, skfold, LOOCV
#X_train = X_train_s.astype('float32')/255.0
#y_train = y_train_s.astype('uint')
#X_val = X_val_s.astype('float32')/255.0
#y_val = y_val_s.astype('uint')

#Optimization Technique
sgd = SGD(learning_rate=0.001)
model.compile(optimizer=sgd,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
#Regularisation Technique 
early_stopping_monitor = EarlyStopping(patience=5)

#Tensorflow Log and Graph generator
NAME ="Project1-Model"
tensorboard = TensorBoard(log_dir='logs_1/{}'.format(NAME))

#Defining callbacks with ALR, Early stopping , Tensorboard
callbacks_list = [early_stopping_monitor,tensorboard,lrate]
#callback with only Early stopping  
#callbacks_list = [early_stopping_monitor]
#callback with only lrate
#callbacks_list = [lrate]
#callback with only tensorboard
#callbacks_list = [tensorboard]

#Fit model using the training and validation dataset
model.fit(X_train, y_train, batch_size=128, epochs=20, validation_data =(X_val,y_val), callbacks =callbacks_list)

In [ ]:
#Datascaling of testing data
X_test = X_test.astype('float32')/255.0
y_test = y_test.astype('uint')

#Run the unseen data using the predicted model to find the test data accuracy
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print('Accuracy: %.3f' % acc)

#Calculating Classification Error 
err=1-acc
print('Classification Error: %.3f' % err)

In [ ]:
#Evaluating Model Performance by creating confusion matrix and classification report
from sklearn.metrics import classification_report, confusion_matrix

y_proba = model.predict(X_test) 
y_pred = model.predict_classes(X_test)
print('Confusion Matrix')
print(confusion_matrix(y_pred, y_test))
print('Classification Report')
print(classification_report(y_pred, y_test))

In [ ]:
#Plotting the Confusion Matrix
def plot_confusion_matrix(cm,
                          target_names,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=True):
    import itertools

    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")


    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.show()

In [ ]:
plot_confusion_matrix(confusion_matrix(y_pred, y_test), 
                      normalize    = False,
                      target_names = ["street","forest","mountain","sea","glacier","buildings"],
                      title        = "Confusion Matrix")